In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from google.cloud import bigquery

In [2]:
#Setup client project id
client = bigquery.Client(project = "aqueous-nebula-343009")

# Creating Load Data Function

In [3]:
def load_data_from_ref_object(client, dataset_name, table_name, max_results = 10):
    dataset_ref = client.dataset(dataset_name)
    dataset = client.get_dataset(dataset_ref)
    
    print("List of all Tables in Dataset --")
    tables = list(client.list_tables(dataset))
    for t in tables:
        print(t.table_id)
        
    table_ref =  dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    
    print("Schema of the Table --")
    print(table.schema)
    data = client.list_rows(table, max_results = max_results).to_dataframe()
    return data

# Testing Out the Function

In [4]:
#Give your Dataset and Table Name
data = load_data_from_ref_object(client,"bitcoin_blockhain", "blocks")
data.head()

# Creating Function to Read to Pandas DataFrame

In [5]:
def load_dataset_from_query(client, query_):
    job = client.query(query_)
    return job.to_dataframe()

# Extracting only the Fields needed for Analysis

In [6]:
QUERY =  "SELECT block_id, previous_block, timestamp, difficultyTarget, work_terahash FROM aqueous-nebula-343009.bitcoin_blockhain.blocks"
data = load_dataset_from_query(client, QUERY )
data.head()

In [7]:
data.sort_values(by = 'timestamp', inplace = True)
data.head(3)

In [8]:
data['timestamp'].min(), data['timestamp'].max()

# More Bitcoin has been Mined

In [9]:
plt.plot(data.work_terahash.values)

# Difficulty to Enter Bitcoin has decreased over the course of time

In [10]:
plt.plot(data.difficultyTarget.values)

In [28]:
QUERY =  "SELECT AVG(difficultyTarget) as AvgDifficultyThreshold, AVG(work_terahash) as TeraHash FROM aqueous-nebula-343009.bitcoin_blockhain.blocks"
data = load_dataset_from_query(client, QUERY )
data.head()

In [30]:
QUERY =  "SELECT AVG(EXTRACT(YEAR from DATE(TIMESTAMP_MILLIS(timestamp)))) AS year, AVG(difficultyTarget) as AvgDifficultyThreshold, AVG(work_terahash) as TeraHash FROM aqueous-nebula-343009.bitcoin_blockhain.blocks"
data = load_dataset_from_query(client, QUERY )
data.head()

In [32]:
QUERY =  "SELECT EXTRACT(YEAR from DATE(TIMESTAMP_MILLIS(timestamp))) AS year, AVG(difficultyTarget) as AvgDifficultyThreshold, AVG(work_terahash) as TeraHash FROM aqueous-nebula-343009.bitcoin_blockhain.blocks GROUP BY year ORDER BY year"
data = load_dataset_from_query(client, QUERY )
data.head()

## Difficulty Threshold has decreased over the years

In [33]:
#Difficulty Threshold in Years
plt.figure(figsize = (8,6))
sns.barplot(data['year'], data['AvgDifficultyThreshold'])

## HashPower has increased over the years especially on Year 2018
This suggests 2018 is the year when more people started learning BitCoin

In [34]:
#HashPower in Years
plt.figure(figsize = (8,6))
sns.barplot(data['year'], data['TeraHash'])